# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [ ]:
# imports

import os
import requests
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI


In [3]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2:1b'

In [ ]:
# set up environment

load_dotenv(override=True)
openrouter_api_key = os.getenv('OPENROUTER_API_KEY')

if openrouter_api_key and openrouter_api_key.startswith('sk-or-') and len(openrouter_api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")



In [5]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""



In [6]:
#prompts
system_prompt = "You are a helpful technical tutor who answers questions about python code, software engineering, data science and LLMs"
user_prompt = "Please give a detailed explanation to the following question: " + question

In [7]:
# messages

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

In [8]:
openrouter_url = "https://openrouter.ai/api/v1"

openrouter= OpenAI(base_url=openrouter_url, api_key=openrouter_api_key)

In [9]:
OLLAMA_BASE_URL = "http://localhost:11434/v1"

ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')

In [ ]:
response=openrouter.chat.completions.create(model=MODEL_GPT,messages=messages)

result=response.choices[0].message.content
print(result)

In [21]:
# streaming function
def stream_response(model_type="openrouter"):
    if model_type == "openrouter":
        client, model = openrouter, MODEL_GPT
    elif model_type == "ollama":
        client, model = ollama, MODEL_LLAMA
    else:
        raise ValueError(f"Unknown model_type '{model_type}'. Choose 'openrouter' or 'ollama'.")

    stream = client.chat.completions.create(model=model, messages=messages, stream=True)

    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [ ]:
# Get gpt-4o-mini to answer, with streaming
stream_response("openrouter")

In [ ]:
# Get Llama 3.2 to answer
requests.get("http://localhost:11434").content

In [ ]:
response=ollama.chat.completions.create(model=MODEL_LLAMA,messages=messages)

result=response.choices[0].message.content
print(result)

In [ ]:
# Get Llama 3.2 to answer, with streaming
stream_response("ollama")

In [ ]:
#human friendly interface
import ipywidgets as widgets
from IPython.display import display, Markdown

SYSTEM_PROMPT = "You are a helpful technical tutor who answers questions about Python code, software engineering, data science and LLMs."

model_toggle = widgets.ToggleButtons(
    options=[("GPT-4o-mini", "openrouter"), ("Llama 3.2", "ollama")],
    value="openrouter",
    description="Model:",
    style=widgets.ToggleButtonsStyle(button_width="120px"),
)

question_box = widgets.Textarea(
    placeholder="Type your technical question here...",
    description="Question:",
    layout=widgets.Layout(width="600px", height="100px"),
    style={"description_width": "initial"},
)

submit_btn   = widgets.Button(description="Ask", button_style="primary")
loader_label = widgets.Label(value="")
output_area  = widgets.Output()

def on_submit(_):
    output_area.clear_output()
    q = question_box.value.strip()
    if not q:
        with output_area:
            print("Please enter a question.")
        return

    loader_label.value    = "⏳ Thinking..."
    submit_btn.disabled   = True

    msgs = [{"role": "system", "content": SYSTEM_PROMPT}, {"role": "user", "content": q}]
    try:
        if model_toggle.value == "openrouter":
            stream = openrouter.chat.completions.create(model=MODEL_GPT, messages=msgs, stream=True)
        else:
            stream = ollama.chat.completions.create(model=MODEL_LLAMA, messages=msgs, stream=True)
        answer = "".join(chunk.choices[0].delta.content or "" for chunk in stream)
    except Exception as e:
        answer = f"**Error:** {e}"

    loader_label.value  = ""
    submit_btn.disabled = False

    with output_area:
        print(f"Model: {model_toggle.label}  |  Question: {q}\n")
        display(Markdown(answer))

submit_btn.on_click(on_submit)

display(model_toggle, question_box, widgets.HBox([submit_btn, loader_label]), output_area)